In [2]:
#| default_exp classes.DomoPublish

In [3]:
# |export
from fastcore.basics import patch_to


In [4]:
#| exporti

from dataclasses import dataclass, field

from typing import Optional

import datetime as dt
import asyncio

# import importlib
# import json
# import uuid
# import time

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.publish as publish_routes

# import Library.DomoClasses.DomoDataset as dmda
# import Library.DomoClasses.DomoLineage as dmdl



In [5]:
# | export
@dataclass
class DomoPublication_Subscription:
    subscription_id: str
    publication_id: str
    domain: str
    created_dt: Optional[dt.datetime] = None

    @classmethod
    def _from_json(cls, json):

        dd = json
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(json)

        return cls(subscription_id=dd.id,
                   publication_id=dd.publicationId,
                   domain=dd.domain,
                   created_dt=dt.datetime.fromtimestamp(dd.created/1000) if dd.created else None)


#### sample implementation of DomoPublication_Subscription

In [11]:
import os
import pandas as pd

publication_id = "431005fd-ec85-41fe-88f6-7b888222aec2"
publication_id = "a1a192c0-75f5-4d1a-bb2b-ff0e6bf67c68"

token_auth = dmda.DomoTokenAuth(
    # domo_instance="domo-dojo",
    # domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]

    domo_instance=os.environ["BETA_INSTANCE"],
    domo_access_token=os.environ["BETA_ACCESS_TOKEN"]
)

res = await publish_routes.get_publication_by_id(publication_id=publication_id,
                                  auth=token_auth)

from pprint import pprint
pprint(res.response)

subscriber_ls = res.response.get('subscriptionAuthorizations')

[DomoPublication_Subscription._from_json(subscriber).__dict__ for subscriber in subscriber_ls]

{'children': [{'children': [],
               'content': {'customerId': '',
                           'domain': 'playstation-beta.domo.com',
                           'domoObjectId': '07e03f4f-eb70-41ee-a7ba-0c8d81843f04',
                           'hash': '462279fc-fd54-485e-bb6a-7e70461ac846',
                           'id': '04170f21-1e72-4279-858c-d0f23aa947ae',
                           'type': 'DATASET',
                           'updated': 1675430674796,
                           'userId': '1360345502'},
               'created': 1675430674796,
               'customerId': '',
               'hasDuplicate': False,
               'id': 'bce5e80d-f114-470e-9863-11388246787b',
               'isBackup': False,
               'isV2': True,
               'subscribers': [],
               'subscriptionAuthorizations': [],
               'type': 'CONTENT',
               'useDirectContent': True}],
 'content': {'customerId': 'mmmm-0012-0350',
             'domain': 'playstation

[{'subscription_id': '4a6d82e3-a7da-4cba-a9a4-fed383bc0f7e',
  'publication_id': 'a1a192c0-75f5-4d1a-bb2b-ff0e6bf67c68',
  'domain': 'playstation-dso-adobe.domo.com',
  'created_dt': datetime.datetime(2022, 11, 22, 20, 41, 34, 914000)}]

In [7]:
# | export
@dataclass
class DomoPublication_Content:
    id: str
    entity_type: str
    entity_id: str
    content_domain: str

    @classmethod
    def _from_json(cls, obj: dict):

        dd = obj
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(obj)

        dmpc = cls(
            id=dd.id,
            entity_type=dd.content.type,
            entity_id=dd.content.domoObjectId,
            content_domain=dd.content.domain)
        
        return dmpc

    def to_json(self):
        temp_dict = {'domain': self.content_domain,
                     'domoObjectId': self.entity_id,
                     'customerId': self.content_domain,
                     'type': self.entity_type}
        return temp_dict


In [8]:
import os
import pandas as pd

publication_id = "431005fd-ec85-41fe-88f6-7b888222aec2"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await publish_routes.get_publication_by_id(publication_id=publication_id,
                                                 auth=token_auth)

content_ls = res.response.get('children')

[DomoPublication_Content._from_json(
    content).__dict__ for content in content_ls]


[{'id': 'ec909f98-b95b-4b04-b328-f21da172822f',
  'entity_type': 'PAGE',
  'entity_id': '1880852133',
  'content_domain': 'domo-dojo.domo.com'},
 {'id': '460edf09-cca0-4a4c-bba0-51f82a1a5e84',
  'entity_type': 'DATASET',
  'entity_id': '8848cdf1-28d4-46e9-93e4-7afb4b5ec1d2',
  'content_domain': 'domo-dojo.domo.com'},
 {'id': 'aba70e14-ecf2-4c20-98af-8ffb4d56d183',
  'entity_type': 'DATASET',
  'entity_id': 'cf149711-df55-4dc3-a159-59a26d066baa',
  'content_domain': 'domo-dojo.domo.com'},
 {'id': 'b70b4e6f-c4a9-444b-886a-774fa5405472',
  'entity_type': 'DATASET',
  'entity_id': 'ee24d234-303e-46aa-b062-9905dc0d8622',
  'content_domain': 'domo-dojo.domo.com'},
 {'id': 'b9683bc2-33cc-4b10-8b31-9e394da85655',
  'entity_type': 'DATASET',
  'entity_id': 'ea859b06-5dbe-4c31-be76-de921447d550',
  'content_domain': 'domo-dojo.domo.com'}]

In [9]:
# | export
@dataclass
class DomoPublication:
    id: str
    name: str
    description: str
    is_v2: bool
    created_dt: dt.datetime
    full_auth: dmda.DomoFullAuth = field(default=None, repr=False)

    subscription_authorizations: [
        DomoPublication_Subscription] = field(default_factory=list)
    content_entity_ls: [DomoPublication_Content] = field(default_factory=list)

    content_page_id_ls: [str] = None
    content_dataset_id_ls: [str] = None

    lineage: dmdl.DomoLineage = None

    def __post_init__(self):
        self.lineage = dmdl.DomoLineage(id=self.id,
                                        parent=self)

    @classmethod
    def _from_json(cls, obj, full_auth: dmda.DomoFullAuth = None):
        dd = DictDot(obj)

        domo_pub = cls(
            id=dd.id,
            name=dd.name,
            description=dd.description,
            created_dt=dt.datetime.fromtimestamp(
                dd.created/1000) if dd.created else None,
            is_v2=dd.isV2,
            full_auth=full_auth
        )

        if dd.subscription_authorizations and len(dd.subscription_authorizations) > 0:
            domo_pub.subscription_authorizations = [DomoPublication_Content._from_json(sub)
                                                    for sub in dd.subscription_authorizations]

        # publish only supports sharing pages and datasets
        if dd.children and len(dd.children) > 0:
            for child in dd.children:
                dmpc = DomoPublication_Content(
                    content_id=child.id,
                    entity_type=child.content.type,
                    entity_id=child.content.domoObjectId,
                    content_domain=child.content.domain)

                if not domo_pub.content_entity_ls:
                    domo_pub.content_entity_ls = []

                domo_pub.content_entity_ls.append(dmpc)

                if dmpc.entity_type == 'PAGE':
                    if not domo_pub.content_page_id_ls:
                        domo_pub.content_page_id_ls = []
                    domo_pub.content_page_id_ls.append(dmpc.entity_id)

                if dmpc.entity_type == 'DATASET':
                    if not domo_pub.content_dataset_id_ls:
                        domo_pub.content_dataset_id_ls = []
                    domo_pub.content_dataset_id_ls.append(dmpc.entity_id)

        return domo_pub


NameError: name 'dmdl' is not defined

In [ ]:
@patch_to(DomoPublication, cls_method = True)
async def get_from_id(cls, publication_id=None, full_auth: dmda.DomoFullAuth = None):

    full_auth = full_auth or cls.full_auth

    publication_id = publication_id or cls.publication_id

    res = await publish_routes.get_publication_by_id(full_auth=full_auth, publication_id=publication_id)

    if res.status != 200:
        print(res)
        return None

    return cls._from_json(obj=res.response, full_auth=full_auth)


In [ ]:


    def convert_content_to_dataframe(self, return_raw: bool = False):

        output_ls = [{'plubication_id': self.id,
                      'publication_name': self.name,
                      'is_v2': self.is_v2,
                      'publish_created_dt': self.created_dt,
                      'entity_type': row.type,
                      'entity_id': row.id
                      } for row in self.content_entity_ls]

        if return_raw:
            return output_ls

        return pd.DataFrame(output_ls)

    def convert_lineage_to_dataframe(self, return_raw: bool = False):
        import pandas as pd
        import re

        flat_lineage_ls = self.lineage._flatten_lineage()

        output_ls = [{'plubication_id': self.id,
                      'publication_name': self.name,
                      'is_v2': self.is_v2,
                      'publish_created_dt': self.created_dt,
                      'entity_type': row.get('entity_type'),
                      'entity_id': row.get('entity_id')
                      } for row in flat_lineage_ls]

        if return_raw:
            return output_ls

        return pd.DataFrame(output_ls)

    @classmethod
    async def create_publication(cls,
                                 name : str,
                                 content_ls : [DomoPublication_Content],
                                 subscription_ls: [DomoPublication_Subscription],
                                 unique_id: str = None,
                                 description: str = None,
                                 full_auth: dmda.DomoFullAuth = None,
                                 debug: bool = False):

        if not isinstance(subscription_ls, list):
            subscription_ls = [subscription_ls]

        full_auth = full_auth or cls.full_auth
        domain_ls = []
        content_json_ls = []
        for sub in subscription_ls:
            domain_ls.append(sub.domain)
        for content_item in content_ls:
            content_json_ls.append(content_item.to_json())

        if not unique_id:
            unique_id = str(uuid.uuid4())
        if not description:
            description = ''

        body = publish_routes.generate_publish_body(url= f'{full_auth.domo_instance}.domo.com',
                                                    sub_domain_ls=domain_ls,
                                                    content_ls= content_json_ls,
                                                    name= name,
                                                    unique_id=unique_id,
                                                    description=description,
                                                    is_new= True)

        res = await publish_routes.create_publish_job(full_auth= full_auth, body=body)
        if debug:
            print('Create the new Publish job')
        if res.status != 200:
            print(res)
            await asyncio.sleep(2)
            res = await publish_routes.get_publication_by_id(full_auth=full_auth, publication_id=unique_id)
            if res.status != 200:
                return None
            else:
                return cls._from_json(obj=res.response, full_auth=full_auth)

        return cls._from_json(obj=res.response, full_auth=full_auth)

    @classmethod
    async def update_publication(cls,
                                 name : str,
                                 content_ls : [DomoPublication_Content],
                                 subscription_ls: [DomoPublication_Subscription],
                                 publication_id: str,
                                 description: str = None,
                                 full_auth: dmda.DomoFullAuth = None,
                                 debug: bool = False):

        if not isinstance(subscription_ls, list):
            subscription_ls = [subscription_ls]

        full_auth = full_auth or cls.full_auth
        domain_ls = []
        content_json_ls = []
        for sub in subscription_ls:
            domain_ls.append(sub.domain)
        for content_item in content_ls:
            content_json_ls.append(content_item.to_json())

        if not description:
            description = ''
        body = publish_routes.generate_publish_body(url= f'{full_auth.domo_instance}.domo.com',
                                                    sub_domain_ls=domain_ls,
                                                    content_ls= content_json_ls,
                                                    name= name,
                                                    unique_id=publication_id,
                                                    description=description,
                                                    is_new= False)

        res = await publish_routes.udpate_publish_job(full_auth = full_auth,
                                                      publication_id= publication_id,
                                                      body= body)
        if debug:
            print('Update Publish job by id')
        if res.status != 200:
            print(res)
            await asyncio.sleep(2)
            res = await publish_routes.get_publication_by_id(full_auth=full_auth, publication_id=publication_id)
            if res.status != 200:
                return None
            else:
                return cls._from_json(obj=res.response, full_auth=full_auth)

        return cls._from_json(obj=res.response, full_auth=full_auth)

    @classmethod
    async def get_subscription_invites_list(cls, full_auth: dmda.DomoFullAuth,
                                            debug: bool = False):

        res = await publish_routes.get_subscription_invites_list(full_auth=full_auth,
                                                                 debug=debug)
        if debug:
            print('Getting Publish subscription invites')

        if res.status == 200:
            return res.response

    @classmethod
    async def accept_invite_by_id(cls,
                                    full_auth: dmda.DomoFullAuth,
                                  subscription_id : str,
                                  debug: bool = False):

        res = await publish_routes.accept_invite_by_id(full_auth=full_auth,
                                                       subscription_id=subscription_id,
                                                       debug=debug)
        if debug:
            print(f'Accept invite by id {subscription_id}')

        if res.status == 200:
            return res.response
